In [3]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [5]:
url = 'https://galmedpharma.investorroom.com/press-releases'
page_url ='https://galmedpharma.investorroom.com/press-releases' 

In [8]:
page_url

'https://galmedpharma.investorroom.com/press-releases'

In [7]:
!pip install selenium

In [9]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException, ElementClickInterceptedException
import time

# Setup the WebDriver for Chrome
options = Options()
options.headless = True

# Initialize WebDriver
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
except Exception as e:
    print(f"Failed to initialize WebDriver: {e}")
    raise

driver.implicitly_wait(10)  # Set implicit wait

# Navigate to the URL
url = "https://galmedpharma.investorroom.com/press-releases"
driver.get(url)

# Define the XPath for the "Load More" button
load_more_xpath = '//*[@id="wd_printable_content"]/div/div[4]/button' 
                  
try:
    while True:
        try:
            # Wait for the "Load More" button to be present
            load_more_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, load_more_xpath)))
            
            # Scroll to the "Load More" button using JavaScript
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
            time.sleep(2)  # Allow time for any animations or overlays to complete

            # Check if the button is clickable and click it
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, load_more_xpath)))
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)  # Allow time for new content to load

        except ElementNotInteractableException:
            print("ElementNotInteractableException: Trying to interact with the button again after scrolling a bit more.")
            driver.execute_script("window.scrollBy(0, 250);")  # Scroll down a bit more
        except ElementClickInterceptedException:
            print("ElementClickInterceptedException: Trying to interact with the button again after scrolling a bit more.")
            driver.execute_script("window.scrollBy(0, 250);")  # Scroll down a bit more
        except TimeoutException:
            print("TimeoutException: Timeout waiting for 'Load More' button.")
            break
        except NoSuchElementException:
            print("NoSuchElementException: No 'Load More' button found.")
            break
except Exception as e:
    print(f"Exception encountered: {e}")
finally:
    # Extract links from the page
    try:
        links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, '#wd_printable_content a')]
        print("Extracted links:", links)
    except Exception as e:
        print(f"Error extracting links: {e}")
    
    # Close the WebDriver
    driver.quit()


TimeoutException: Timeout waiting for 'Load More' button.
Extracted links: ['https://galmedpharma.investorroom.com/press-releases#', 'https://galmedpharma.investorroom.com/email-alerts?type=1', 'https://galmedpharma.investorroom.com/press-releases#', 'https://galmedpharma.investorroom.com/2024-04-04-Galmed-Pharmaceuticals-Ltd-Files-Annual-Report-on-Form-20-F-for-the-Fiscal-Year-Ended-December-31,-2023', 'https://galmedpharma.investorroom.com/2024-04-04-Galmed-Pharmaceuticals-Ltd-Files-Annual-Report-on-Form-20-F-for-the-Fiscal-Year-Ended-December-31,-2023?asPDF', 'https://galmedpharma.investorroom.com/2024-04-04-Galmed-Pharmaceuticals-Ltd-Files-Annual-Report-on-Form-20-F-for-the-Fiscal-Year-Ended-December-31,-2023', 'https://galmedpharma.investorroom.com/2024-04-04-Galmed-Pharmaceuticals-Ltd-Files-Annual-Report-on-Form-20-F-for-the-Fiscal-Year-Ended-December-31,-2023#assets_43_188-3', 'https://galmedpharma.investorroom.com/2024-03-15-Galmed-Announces-Grant-of-New-Patent-for-the-Combinat

In [19]:
actual_list = []
for i in links:
    if ("asPDF" not in i) and ("#asset" not in i):
        actual_list.append(i)

all_links = set(actual_list)


In [20]:
all_links

{'https://galmedpharma.investorroom.com/2011-12-21-CH-Health-invests-in-Galmed-Haim-Hurvitz-appointed-Chairman',
 'https://galmedpharma.investorroom.com/2012-03-12-Phase-II-Aramchol-Clinical-Trial-Results',
 'https://galmedpharma.investorroom.com/2012-12-07-International-Conference-in-Memory-of-the-Late-Professor-Tuvia-Gilat',
 'https://galmedpharma.investorroom.com/2014-03-12-Galmed-Pharmaceuticals-Announces-Pricing-of-Initial-Public-Offering',
 'https://galmedpharma.investorroom.com/2014-03-18-Galmed-Pharmaceuticals-Announces-Closing-of-Initial-Public-Offering-and-Full-Exercise-of-Underwriters-Option-to-Purchase-Additional-Shares',
 'https://galmedpharma.investorroom.com/2014-05-15-Galmed-Pharmaceuticals-Reports-First-Quarter-2014-Financial-Results',
 'https://galmedpharma.investorroom.com/2014-06-09-Galmed-Pharmaceuticals-Announces-the-Filing-of-a-Provisional-Patent-Application-for-the-Use-of-Aramchol-for-the-Treatment-of-Lipodystrophy',
 'https://galmedpharma.investorroom.com/2014-

In [21]:
# List of links to be removed
links_to_remove = [
    'https://galmedpharma.investorroom.com/email-alerts',
    'https://galmedpharma.investorroom.com/email-alerts?type=1',
    'https://galmedpharma.investorroom.com/index.php?s=95&rsspage=43',
    'https://galmedpharma.investorroom.com/press-releases#',
    'https://galmedpharma.investorroom.com/press-releases#email',
    'https://galmedpharma.investorroom.com/press-releases#print',
    'https://twitter.com/share?url=https%3A%2F%2Fgalmedpharma.investorroom.com%2Findex.php%3Fs%3D43',
    'https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Fgalmedpharma.investorroom.com%2Findex.php%3Fs%3D43',
    'https://www.linkedin.com/shareArticle?mini=true&url=https%3A%2F%2Fgalmedpharma.investorroom.com%2Findex.php%3Fs%3D43'

]
filtered_links = []
# Filter out the links to remove
filtered_links = [link for link in all_links if link not in links_to_remove]

final_filtered_links=set(filtered_links)


In [22]:
print(final_filtered_links)

{'https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome', 'https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol', 'https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC', 'https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update', 'https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares', 'https://galmedpharma.investorroom.com/press-releases?item=20', 'https://galmedpharma.investorroom.com/2023-06-01-Galmed-Pharmaceuticals-Reports-First-Quarter-2023-Financial-Results-and-Recent-Developments', 'https:

In [23]:
len(final_filtered_links)

169

In [30]:
import re

# Function to extract the date from the URL
def extract_date_from_url(link):
    match = re.search(r'/(\d{4}-\d{2}-\d{2})-', link)
    if match:
        return match.group(1)
    return "Date not found"

# List to hold the URLs with dates
dated_links = []

# Process each URL
for link in final_filtered_links:
    date = extract_date_from_url(link)
    dated_links.append(f"{date} - {link}")

# Print the result
for dated_link in dated_links:
    print(dated_link)


2016-06-30 - https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome
2020-11-09 - https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol
2023-07-10 - https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC
2017-11-09 - https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update
2018-06-18 - https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares
Date not found - https://galmedpharma.investorroom.com/press-releases?item=20
2023-06-01 - https://galmedpharma.investorroom.com/2023-06-01-Galmed-Pharmaceuticals-Rep

In [32]:
len(dated_links)

169

In [34]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

# List of URLs
links = [
    
    "https://galmedpharma.investorroom.com/press-releases?item=20"
    
]
dated_links1=[]

def fetch_date_from_link(url):
    # Send a GET request to the webpage
    response = requests.get(url)
    
    # Parse the content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the date in the webpage content
    date = None
    
    # Attempt to locate the date by inspecting common date container tags
    for tag in ['time', 'span', 'p', 'div']:
        for class_name in ['date', 'published-date', 'pub-date', 'release-date', 'datetime']:
            element = soup.find(tag, class_=class_name)
            if element:
                date = element.text.strip()
                break
        if date:
            break
    
    if not date:
        # As a fallback, search for date patterns in text
        date_patterns = [
            r'\b\w{3,9}\s\d{1,2},\s\d{4}\b',  # e.g., June 4, 2014
            r'\b\d{1,2}\s\w{3,9}\s\d{4}\b'    # e.g., 4 June 2014
        ]
        for pattern in date_patterns:
            match = re.search(pattern, soup.text)
            if match:
                date = match.group()
                break
    
    # Convert the date to YYYY-MM-DD format if found
    if date:
        try:
            date_obj = datetime.strptime(date, "%B %d, %Y")
        except ValueError:
            date_obj = datetime.strptime(date, "%d %B %Y")
        date_formatted = date_obj.strftime("%Y-%m-%d")
    else:
        date_formatted = "Date not found"
    
    return date_formatted

# Fetch dates for all links
for link in links:
    date = fetch_date_from_link(link)
    # print(f"Date {date} - Link: {link}")
    dated_links1.append(f"{date} - {link}")


In [35]:
dated_links1

['2014-04-28 - https://galmedpharma.investorroom.com/press-releases?item=20']

In [38]:
links = ["https://galmedpharma.investorroom.com/press-releases?item=99"]
dated_links2 = []
text = "TEL AVIV, Israel, Feb. 7, 2018 /PRNewswire/ -- Galmed"

# Find the position of the date in the text
date_index = text.find("Feb. 7, 2018")

if date_index != -1:
    # Extract the date using slicing
    date = text[date_index:date_index + len("Feb. 7, 2018")]

    # Reformat the date if needed
    formatted_date = date.replace("Feb. 7, 2018", "2018-02-07")

    # Associate the date with the link
    for link in links:
        dated_links2.append(f"{formatted_date} - {link}")

    # print("Date:", formatted_date)
    # print("Link:", link)
else:
    print("Date not found in the text.")

# print("dated_links2:", dated_links2)


In [40]:
dated_links2

['2018-02-07 - https://galmedpharma.investorroom.com/press-releases?item=99']

In [42]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
dated_links3=[]
# URL of the webpage
url = "https://galmedpharma.investorroom.com/Recently-Published-Independent-Research-Supports-the-Importance-of-Reverse-Cholesterol-Transport-on-NASH-Pathogenesis"

# Send a GET request to the webpage
response = requests.get(url)

# Parse the content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# Find the date in the webpage content
date = None

# Attempt to locate the date by inspecting common date container tags
for tag in ['time', 'span', 'p', 'div']:
    for class_name in ['date', 'published-date', 'pub-date', 'release-date', 'datetime']:
        element = soup.find(tag, class_=class_name)
        if element:
            date = element.text.strip()
            break
    if date:
        break

if not date:
    # As a fallback, search for date patterns in text
    date_patterns = [
        r'\b\w{3,9}\s\d{1,2},\s\d{4}\b',  # e.g., June 4, 2014
        r'\b\d{1,2}\s\w{3,9}\s\d{4}\b'    # e.g., 4 June 2014
    ]
    for pattern in date_patterns:
        match = re.search(pattern, soup.text)
        if match:
            date = match.group()
            break

dated_links2 = []
# Convert the date to YYYY-MM-DD format if found
if date:
    try:
        date_obj = datetime.strptime(date, "%B %d, %Y")
    except ValueError:
        date_obj = datetime.strptime(date, "%d %B %Y")
    date_formatted = date_obj.strftime("%Y-%m-%d")
else:
    date_formatted = "Date not found"

# print(f"{date_formatted} - Link: {url}")
dated_links2.append(f"{date_formatted} - {url}")
print(dated_links2)


['2014-06-04 - https://galmedpharma.investorroom.com/Recently-Published-Independent-Research-Supports-the-Importance-of-Reverse-Cholesterol-Transport-on-NASH-Pathogenesis']


In [44]:
# Define the URLs
# links_1 = [
#     'https://galmedpharma.investorroom.com/2020-05-07-Galmed-Pharmaceuticals-to-Report-First-Quarter-2020-Financial-Results-and-Provide-Business-Update-on-Thursday-May-14',
#     'https://galmedpharma.investorroom.com/2014-07-22-Galmed-Announces-FDA-Clearance-of-IND-of-Armachol-for-the-Treatment-of-Fatty-Liver-Disorders',
#     'https://galmedpharma.investorroom.com/2022-06-17-Galmed-Pharmaceuticals-Announces-Receipt-of-Nasdaq-Minimum-Bid-Price-Notification',
#     'https://galmedpharma.investorroom.com/2018-06-11-Galmed-Pharmaceuticals-to-Host-Conference-Call-to-Discuss-Top-Line-52-Week-Results-from-the-Global-Phase-2b-ARREST-Study',
#     'https://galmedpharma.investorroom.com/2014-11-05-Galmed-Pharmaceuticals-Third-Quarter-2014-Conference-Call-and-Webcast-Scheduled-for-Wednesday-November-12-2014'
# ]

# links_2 = [
#     'https://galmedpharma.investorroom.com/2016-05-03-Mount-Sinai-and-Galmed-Pharmaceuticals-to-Collaborate-in-an-Investigator-Initiated-Phase-IIa-Trial-to-Evaluate-the-Effect-of-AramcholTM-in-Combination-with-Vitamin-D-for-the-Treatment-of-Patients-with-Fibrotic-Nonalcoholic-Fatty-Liver-Disease',
#     'https://galmedpharma.investorroom.com/2017-07-31-Galmed-Pharmaceuticals-Reports-Second-Quarter-2017-Financial-Results-and-Provides-Business-Update',
#     'https://galmedpharma.investorroom.com/2015-05-13-Galmed-Pharmaceuticals-Reports-First-Quarter-2015-Financial-Results-and-Provides-Business-Update',
#     'https://galmedpharma.investorroom.com/2016-03-15-Galmed-Pharmaceuticals-Fourth-Quarter-2015-Conference-Call-and-Webcast-Scheduled-for-Tuesday-March-22nd-2016',
#     'https://galmedpharma.investorroom.com/2018-02-08-Galmed-Pharmaceuticals-to-Provide-Corporate-Update-at-the-LEERINK-Partners-7th-Annual-Global-Healthcare-Conference'
# ]

# print("dated_links2:", dated_links2)


# Combine the lists
combined_links = dated_links + dated_links1 + dated_links2 + dated_links3

# Output the combined list
print(combined_links)


['2016-06-30 - https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome', '2020-11-09 - https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol', '2023-07-10 - https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC', '2017-11-09 - https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update', '2018-06-18 - https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares', 'Date not found - https://galmedpharma.investorroom.com/press-releases?item=20', '2023-06-01 - https://galmedpharma.investorroom.com/2023-06-01-Galmed

In [46]:
len((combined_links))

171

In [48]:
links_to_remove1 = [
    'Date not found - https://galmedpharma.investorroom.com/press-releases?item=20',
    'Date not found - https://galmedpharma.investorroom.com/press-releases?item=94',
    'Date not found - https://galmedpharma.investorroom.com/press-releases?item=99',
    'Date not found - https://galmedpharma.investorroom.com/Recently-Published-Independent-Research-Supports-the-Importance-of-Reverse-Cholesterol-Transport-on-NASH-Pathogenesis'
]

final_filtered_links1 = []

# Filter out the links to remove
final_filtered_links1 = [link for link in combined_links if link not in links_to_remove1]

for link in final_filtered_links1:
    # Split each link by " - " to separate date and link
    date, url = link.split(" - ", 1)
    print("Date:", date)
    print("Link:", url)


Date: 2016-06-30
Link: https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome
Date: 2020-11-09
Link: https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol
Date: 2023-07-10
Link: https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC
Date: 2017-11-09
Link: https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update
Date: 2018-06-18
Link: https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares
Date: 2023-06-01
Link: https://galmedpharma.investorroom.com/2023-06-01-Galmed-Pharmaceuticals-Reports-First-Quarter

In [50]:
links = ["https://galmedpharma.investorroom.com/press-releases?item=99"]
dated_links2 = []
text = "TEL AVIV, Israel, Feb. 7, 2018 /PRNewswire/ -- Galmed"

# Find the position of the date in the text
date_index = text.find("Feb. 7, 2018")

if date_index != -1:
    # Extract the date using slicing
    date = text[date_index:date_index + len("Feb. 7, 2018")]

    # Reformat the date if needed
    formatted_date = date.replace("Feb. 7, 2018", "2018-02-07")

    # Associate the date with the link
    for link in links:
        dated_links2.append(f"{formatted_date} - {link}")

    # print("Date:", formatted_date)
    # print("Link:", link)
else:
    print("Date not found in the text.")

# print("dated_links2:", dated_links2)
final_filtered_links2=final_filtered_links1+dated_links2


In [52]:
final_filtered_links2

['2016-06-30 - https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome',
 '2020-11-09 - https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol',
 '2023-07-10 - https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC',
 '2017-11-09 - https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update',
 '2018-06-18 - https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares',
 '2023-06-01 - https://galmedpharma.investorroom.com/2023-06-01-Galmed-Pharmaceuticals-Reports-First-Quarter-2023-Financial-Results-and-Recent-Dev

In [54]:
len(final_filtered_links2)

168

In [56]:
# links_to_remove1 = [
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=20',
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=94',
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=99',
#     'Date not found - https://galmedpharma.investorroom.com/Recently-Published-Independent-Research-Supports-the-Importance-of-Reverse-Cholesterol-Transport-on-NASH-Pathogenesis'

# ]

# final_filtered_links1 = []

# # Filter out the links to remove
# final_filtered_links1 = [link for link in combined_links if link not in links_to_remove1]

dates = []
urls = []

for link in final_filtered_links2:
    # Split each link by " - " to separate date and link
    date, url = link.split(" - ", 1)
    dates.append(date)
    urls.append(url)

print("Dates:", dates)
print("URLs:", urls)


Dates: ['2016-06-30', '2020-11-09', '2023-07-10', '2017-11-09', '2018-06-18', '2023-06-01', '2020-05-07', '2018-09-27', '2015-08-13', '2016-03-22', '2022-04-28', '2015-05-13', '2022-05-02', '2014-03-18', '2021-03-11', '2019-10-31', '2022-07-07', '2016-07-27', '2019-11-06', '2019-07-29', '2021-11-01', '2021-03-16', '2019-10-30', '2021-07-29', '2016-03-01', '2018-10-29', '2015-01-06', '2016-06-07', '2017-10-19', '2018-05-09', '2020-09-10', '2020-11-12', '2016-09-22', '2018-06-19', '2017-01-09', '2015-01-28', '2018-03-13', '2020-12-17', '2016-10-31', '2022-08-04', '2019-04-09', '2017-06-08', '2016-08-03', '2015-11-02', '2015-01-05', '2016-03-07', '2015-05-06', '2023-03-29', '2019-05-07', '2014-07-21', '2014-09-02', '2016-09-29', '2019-04-30', '2014-03-12', '2018-03-05', '2016-01-05', '2014-12-01', '2018-10-02', '2022-01-10', '2018-01-02', '2016-02-11', '2014-11-13', '2015-04-01', '2018-11-05', '2023-01-04', '2021-11-08', '2021-08-19', '2016-03-15', '2017-03-23', '2021-01-26', '2024-03-15'

In [58]:
len(dates)

168

In [60]:
dates

['2016-06-30',
 '2020-11-09',
 '2023-07-10',
 '2017-11-09',
 '2018-06-18',
 '2023-06-01',
 '2020-05-07',
 '2018-09-27',
 '2015-08-13',
 '2016-03-22',
 '2022-04-28',
 '2015-05-13',
 '2022-05-02',
 '2014-03-18',
 '2021-03-11',
 '2019-10-31',
 '2022-07-07',
 '2016-07-27',
 '2019-11-06',
 '2019-07-29',
 '2021-11-01',
 '2021-03-16',
 '2019-10-30',
 '2021-07-29',
 '2016-03-01',
 '2018-10-29',
 '2015-01-06',
 '2016-06-07',
 '2017-10-19',
 '2018-05-09',
 '2020-09-10',
 '2020-11-12',
 '2016-09-22',
 '2018-06-19',
 '2017-01-09',
 '2015-01-28',
 '2018-03-13',
 '2020-12-17',
 '2016-10-31',
 '2022-08-04',
 '2019-04-09',
 '2017-06-08',
 '2016-08-03',
 '2015-11-02',
 '2015-01-05',
 '2016-03-07',
 '2015-05-06',
 '2023-03-29',
 '2019-05-07',
 '2014-07-21',
 '2014-09-02',
 '2016-09-29',
 '2019-04-30',
 '2014-03-12',
 '2018-03-05',
 '2016-01-05',
 '2014-12-01',
 '2018-10-02',
 '2022-01-10',
 '2018-01-02',
 '2016-02-11',
 '2014-11-13',
 '2015-04-01',
 '2018-11-05',
 '2023-01-04',
 '2021-11-08',
 '2021-08-

In [62]:
len(urls)

168

In [64]:
urls

['https://galmedpharma.investorroom.com/2016-06-30-Galmed-Pharmaceuticals-and-Weizmann-Institute-of-Science-to-enter-a-research-collaboration-agreement-to-assess-Aramchol-effects-on-microbiome',
 'https://galmedpharma.investorroom.com/2020-11-09-Galmed-and-MyBiotics-to-Collaborate-in-Development-of-Bespoke-Microbiome-Signature-for-Aramchol',
 'https://galmedpharma.investorroom.com/2023-07-10-Galmed-to-employ-OnKais-disruptive-Artificial-Intelligence-AI-in-its-Planned-Phase-2a-Clinical-Trial-in-PSC',
 'https://galmedpharma.investorroom.com/2017-11-09-Galmed-Pharmaceuticals-Reports-Third-Quarter-2017-Financial-Results-and-Provides-Business-Update',
 'https://galmedpharma.investorroom.com/2018-06-18-Galmed-Pharmaceuticals-Ltd-Announces-Proposed-Public-Offering-of-Ordinary-Shares',
 'https://galmedpharma.investorroom.com/2023-06-01-Galmed-Pharmaceuticals-Reports-First-Quarter-2023-Financial-Results-and-Recent-Developments',
 'https://galmedpharma.investorroom.com/2020-05-07-Galmed-Pharmace

In [44]:
pip install requests beautifulsoup4 pandas

In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the skip_sections function
def skip_sections(text, skip_phrases):
    for phrase in skip_phrases:
        if phrase in text:
            text = text.split(phrase)[0]  # Keep only the content before the skip phrase
    return text.strip()

# Column names for the DataFrame
column_names = ['Author', 'Heading', 'Content', 'Company']

# CSV file path (commented out for this example)
# csv_file_path = r"C:\Users\OMS-DESKTOP\punam\web_scraping\Galmed19.csv"

# Initialize an empty list to store all data
all_data = []

# # List of URLs to scrape
# urls = [
#     "https://galmedpharma.investorroom.com/press-releases?item=20"
# ]

# Define the phrase to skip sections in the content
skip_phrases = ["About Galmed Pharmaceuticals Ltd.","About Galmed Pharmaceuticals Ltd.:"]

# Loop through each link in the list
for url in urls:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        html_content = response.text

        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')
        section = soup.find('section', class_='wd_main_content wd_sec')

        if section is None:
            raise AttributeError("Section with class 'wd_main_content wd_sec' not found")

        heading = section.find('div', class_='wd_title').text.strip()
        company = 'Galmed Pharmaceuticals'

        # Extract content from paragraphs
        content = []
        for element in section.find_all(['p']):
            paragraph_text = element.text.strip()
            content.append(paragraph_text)
        
        # Join content and apply skip_sections
        content = ' '.join(content)
        content = skip_sections(content, skip_phrases)

        # Create the data list
        data_list = ['NA', heading, content, company]  # Initialize data_list with default values        

        # Append the data_list to all_data
        all_data.append(data_list)
        
        # Debug print to check data extraction
        print("Heading:", heading)
        print("Content:", content)

    except requests.exceptions.RequestException as e:
        with open(r'C:\Users\OMS-DESKTOP\punam\web_scraping\error_log.txt', 'a') as file:
            file.write(f"Exception: {str(e)}\n")
            file.write(f"Error in link: {url}\n")
    except AttributeError as e:
        with open(r'C:\Users\OMS-DESKTOP\punam\web_scraping\error_log.txt', 'a') as file:
            file.write(f"Exception: {str(e)}\n")
            file.write(f"Error in link: {url}\n")

# Create DataFrame from all_data
df2 = pd.DataFrame(all_data, columns=column_names)
print(df2)

# Save DataFrame to CSV file (commented out for this example)
# df2.to_csv(csv_file_path, index=False, encoding='utf-8')

# print(f"Data from {len(urls)} links has been saved to {csv_file_path}")


Heading: Galmed Announces Allowance of New Patent for Aramchol for the Treatment of Pulmonary and Dermal Fibrosis
Content: TEL AVIV, Israel, Sept. 26, 2023 /PRNewswire/ -- Galmed Pharmaceuticals Ltd. (Nasdaq: GLMD) ("Galmed" or the "Company"), a clinical-stage biopharmaceutical company for liver, metabolic and fibrotic diseases announced today the allowance of a Japanese patent related to treatment of pulmonary and dermal fibrosis. A similar patent was already granted in Mexico. The approval of the patent in the US and the rest of the world is pending. With this latest patent, Galmed is strengthening and extending the patent protection of its lead compound, Aramchol, until November 2037.   Previously, Galmed reported results showing significant anti-fibrotic effects of Aramchol in a pre-clinical model of lung fibrosis. Treatment with Aramchol resulted in statistically significant fibrosis improvement in a validated bleomycin model of lung fibrosis (IPF), comparable to Pirfenidone which

In [85]:
combined_data1 = pd.DataFrame(all_data, columns=column_names)
print(combined_data1)

    Author                                            Heading  \
0       NA  Galmed Announces Allowance of New Patent for A...   
1       NA  Galmed Reports Significant Anti-Fibrotic Effec...   
2       NA  Galmed Pharmaceuticals Randomizes 120th Patien...   
3       NA  Galmed Pharmaceuticals Completes Two Chronic T...   
4       NA  Aramchol™ Demonstrates Significant Anti-Fibrot...   
..     ...                                                ...   
162     NA           Phase II Aramchol Clinical Trial Results   
163     NA  Galmed Pharmaceuticals Announces Closing of In...   
164     NA  Galmed Pharmaceuticals Commences Patient Scree...   
165     NA  Recently Published Independent Research Suppor...   
166     NA  Galmed Granted "Preferred Technological Enterp...   

                                               Content                 Company  
0    TEL AVIV, Israel, Sept. 26, 2023 /PRNewswire/ ...  Galmed Pharmaceuticals  
1    TEL AVIV, Israel, July 7, 2022 /PRNewswire/ --...  G

In [87]:
import pandas as pd

# links_to_remove1 = [
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=20',
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=94',
#     'Date not found - https://galmedpharma.investorroom.com/press-releases?item=99',
#     'Date not found - https://galmedpharma.investorroom.com/Recently-Published-Independent-Research-Supports-the-Importance-of-Reverse-Cholesterol-Transport-on-NASH-Pathogenesis'

# ]

# final_filtered_links1 = []

# # Filter out the links to remove
# final_filtered_links1 = [link for link in combined_links if link not in links_to_remove1]

dates = []
urls = []

for link in final_filtered_links2:
    # Split each link by " - " to separate date and link
    date, url = link.split(" - ", 1)
    dates.append(date)
    urls.append(url)

# Create a DataFrame
df = pd.DataFrame({'Date': dates, 'Link': urls})

print(df)


           Date                                               Link
0    2023-09-26  https://galmedpharma.investorroom.com/2023-09-...
1    2022-07-07  https://galmedpharma.investorroom.com/2022-07-...
2    2016-06-01  https://galmedpharma.investorroom.com/2016-06-...
3    2014-09-02  https://galmedpharma.investorroom.com/2014-09-...
4    2016-03-30  https://galmedpharma.investorroom.com/2016-03-...
..          ...                                                ...
163  2012-03-12  https://galmedpharma.investorroom.com/2012-03-...
164  2014-03-18  https://galmedpharma.investorroom.com/2014-03-...
165  2014-04-28  https://galmedpharma.investorroom.com/press-re...
166  2014-06-04  https://galmedpharma.investorroom.com/Recently...
167  2018-02-07  https://galmedpharma.investorroom.com/press-re...

[168 rows x 2 columns]


In [79]:
import pandas as pd
csv_file_path = r"C:\Users\OMS-DESKTOP\punam\web_scraping\Galmed43.csv"

# Assuming all_data is a list of lists or a 2D array containing your data
# Assuming column_names is a list containing column names for your DataFrame
# Assuming df is the DataFrame containing dates and links

# Create DataFrame from all_data
combined_data1 = pd.DataFrame(all_data, columns=column_names[:5])  # Adjust columns to match the number of columns in all_data

# Combine the two DataFrames
combined_df = pd.concat([df, combined_data1], axis=1)

# Display the combined DataFrame
print(combined_df)
combined_df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"Data from {len(all_links)} links has been saved to {csv_file_path}")


           Date                                               Link Author  \
0    2023-09-26  https://galmedpharma.investorroom.com/2023-09-...     NA   
1    2022-07-07  https://galmedpharma.investorroom.com/2022-07-...     NA   
2    2016-06-01  https://galmedpharma.investorroom.com/2016-06-...     NA   
3    2014-09-02  https://galmedpharma.investorroom.com/2014-09-...     NA   
4    2016-03-30  https://galmedpharma.investorroom.com/2016-03-...     NA   
..          ...                                                ...    ...   
163  2012-03-12  https://galmedpharma.investorroom.com/2012-03-...     NA   
164  2014-03-18  https://galmedpharma.investorroom.com/2014-03-...     NA   
165  2014-04-28  https://galmedpharma.investorroom.com/press-re...    NaN   
166  2014-06-04  https://galmedpharma.investorroom.com/Recently...    NaN   
167  2018-02-07  https://galmedpharma.investorroom.com/press-re...    NaN   

                                               Heading  \
0    Galmed Annou